# InGaN/GaN PL analysis

In [4]:
%matplotlib qt
import numpy as np
import hyperspy.api as hs
import hyperspy_gui_ipywidgets
import seaborn as sns
import os
from spec_im import PLSpectralImage
from spec_im import plot_si_bands, plot_pl_summary, gui_fname, plot_bss_results, plot_decomp_results
import matplotlib.pyplot as plt

sns.set()
sns.set_style('ticks')

hs.preferences.gui(toolkit="traitsui")

In [ ]:
plt.subplots_adjust(top=0.9)

## Loading data from file

In [5]:
# fname = gui_fname()
fname = 'D:/Chris/BBOD_Share/uv_microscope/548 uGaN qw 5um 3 cryo/190422_171223_andor_asi_hyperspec_scan.h5'
fpath = os.path.dirname(fname)
fbase = os.path.splitext(fname)[0]
sample = os.path.basename(fname)

In [6]:
(os.path.dirname(fname),os.path.basename(fname))
si = PLSpectralImage(fname=fname)

Load from D:/Chris/BBOD_Share/uv_microscope/548 uGaN qw 5um 3 cryo/190422_171223_andor_asi_hyperspec_scan.h5 complete.
1 x 51 x 51 spatial x 2000 spectral points


In [ ]:
si.plot_spec()

In [ ]:
si.set_background(lims=[600,900]) 

In [7]:
si = si[354:700]

In [8]:
esi = si.to_energy()

Interpolating spectral data from nm to eV
354.10366588180545 to 638.0430830917055 nm
51 x 51 spatial x 1927 spectral points


## Summary

In [9]:
f0 = plot_pl_summary(si, show_axes=False, show_scalebar=True, scalebar_alpha=0.5)
plt.subplots_adjust(top=0.9)
f01 = plot_pl_summary(esi,show_axes=False, show_scalebar=True, scalebar_alpha=0.5)
plt.subplots_adjust(top=0.9)

In [ ]:
plt.subplots_adjust(top=0.8)

## Plot data vs eV

In [ ]:
images = []
labels = []
for kk in range(10):
    img = s.isig[(2.5+kk*0.1):(2.6+kk*0.1)].sum(axis=-1).transpose()
    images.append(img)
    labels.append('%0.1f to %0.1f eV' % ((2.5+kk*0.1), (2.6+kk*0.1)))
hs.plot.plot_images(images, labelwrap=20, cmap='viridis', per_row=5, label=labels, suptitle=si.name, axes_decor='off', scalebar=[0,])

In [10]:
f1 = plot_si_bands(esi, 
            (2.5,2.6,'viridis',''), (2.6,2.7,'viridis',''), (2.7,2.8,'viridis',''), (2.8,2.9,'viridis',''), (2.9,3.0,'viridis',''),
            (3.0,3.1,'viridis',''), (3.1,3.2,'viridis',''), (3.2,3.3,'viridis',''), (3.3,3.4,'viridis',''), (3.4,3.5,'viridis',''),
            no_of_cols=5, percentile=5, show_axes=False, show_scalebar=True, scalebar_alpha=0.5)
f1.set_size_inches(10,7)
f1.tight_layout(h_pad=1)

## Hyperspy signal setup

In [11]:
# plot an interactive map with spectra at each location
s = esi.to_signal()
s.plot(navigator_kwds={'cmap': 'viridis'})
# roi = hs.roi.RectangularROI(left=0, right=100., top=0, bottom=100.)
# scrop = roi.interactive(s)

In [ ]:
(s.sum(axis=-1) > 0.9e9).plot(navigator_kwds={'cmap': 'viridis'})

In [ ]:
mask = s.sum(axis=-1) < 0.9e9
mask

In [13]:
s.spikes_removal_tool()

In [ ]:
scrop.plot()

In [ ]:
s = scrop

In [ ]:
s

## PCA with SVD, BSS

In [14]:
# perform principal component analysis, look at the explained variance
s.decomposition(algorithm='svd',)
s.plot_explained_variance_ratio()

In [15]:
s.plot_decomposition_results()

In [16]:
COMPS = 16

In [17]:
# take a look at orthogonalized 
plot_bss_results(s, esi, title='BSS', cmap='viridis', fig_rows=4, components=COMPS, max_iter=2000, 
                show_axes=False, show_scalebar=True, scalebar_alpha=0.5)
# s.blind_source_separation(number_of_components=COMPS, max_iter=2000, on_loadings=True)
# hs.plot.plot_images(s.get_bss_loadings(), labelwrap=20, cmap='viridis', per_row=1, suptitle='BSS loadings of %s' % si.name, axes_decor='off', scalebar=[0,])
# hs.plot.plot_spectra(s.get_bss_factors(), style='mosaic', legend='auto')

[<Figure size 1000x1000 with 12 Axes>,
 <Figure size 1000x1000 with 12 Axes>,
 <Figure size 1000x1000 with 12 Axes>,
 <Figure size 1000x1000 with 12 Axes>]

In [ ]:
list(s.get_decomposition_factors())[0].data.shape

In [ ]:
si.spec_im.shape

## PCA with NMF

In [ ]:
plot_decomp_results(s, esi, title='NMF', cmap='viridis', fig_rows=4, algorithm='nmf', output_dimension=COMPS,
                   show_axes=False, show_scalebar=True, scalebar_alpha=0.5)
#s.decomposition(algorithm='nmf', output_dimension=COMPS)
#hs.plot.plot_images(s.get_decomposition_loadings())
#hs.plot.plot_spectra(s.get_decomposition_factors(), style='mosaic')

In [ ]:
s.plot_decomposition_results()

In [ ]:
sig_list = []
load_list = list(s.get_decomposition_loadings())
factor_list = list(s.get_decomposition_factors())

for kk in range(COMPS):
    sig_list.append(load_list[kk])
    sig_list.append(factor_list[kk])

hs.plot.plot_signals(sig_list)

## Save PDF

In [ ]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages(fbase+"_decomp.pdf")
for fig in range(1, plt.figure().number): ## will open an empty extra figure :(
    pdf.savefig( fig )
pdf.close()
plt.close(fig='all')

In [ ]:
print('test')

In [ ]:
sc = s.get_decomposition_model()
sc.plot()

In [ ]:
(s-sc).plot()

In [ ]:
m = sc.create_model()

In [ ]:
bkg = hs.model.components1D.Offset(offset=3.2e6) 
bkg.offset.bmin = 0.0
bkg.offset.bmax = 1e8
bkg.offset.bounded = True
gan1 = hs.model.components1D.Lorentzian(centre=3.4,)
gan1.A.value = 2e7
gan1.A.bmin = 0.0
gan1.A.bmax = 1e8
gan1.A.bounded = True
gan1.centre.bmin = gan1.centre.value - 0.05
gan1.centre.bmax = gan1.centre.value + 0.05
gan1.centre.bounded = True
gan1.gamma.bmin = -10
gan1.gamma.bmax = 20.0
gan1.gamma.bounded = True
gan1.gamma.value = 0.1
gan2 = hs.model.components1D.Lorentzian(centre=3.25,)
gan2.A.value = 2e7
gan2.A.bmin = -10
gan2.A.bmax = 1e8
gan2.A.bounded = True
gan2.centre.bmin = gan2.centre.value - 0.05
gan2.centre.bmax = gan2.centre.value + 0.05
gan2.centre.bounded = True
gan2.gamma.bmin = -10
gan2.gamma.bmax = 20.0
gan2.gamma.bounded = True
gan2.gamma.value = 0.1
ingan1 = hs.model.components1D.Lorentzian(centre=3.06,)
ingan1.A.value = 2e7
ingan1.A.bmin = -10
ingan1.A.bmax = 1e8
ingan1.A.bounded = True
ingan1.centre.bmin = ingan1.centre.value - 0.05
ingan1.centre.bmax = ingan1.centre.value + 0.05
ingan1.centre.bounded = True
ingan1.gamma.value = 0.1
ingan1.gamma.bmin = -10
ingan1.gamma.bmax = 20.0
ingan1.gamma.bounded = True
ingan2 = hs.model.components1D.Lorentzian(centre=2.89,)
ingan2.A.value = 2e7
ingan2.A.bmin = 0.0
ingan2.A.bmax = 1e8
ingan2.A.bounded = True
ingan2.centre.bmin = ingan2.centre.value - 0.05
ingan2.centre.bmax = ingan2.centre.value + 0.05
ingan2.centre.bounded = True
ingan2.gamma.value = 0.1
ingan2.gamma.bmin = -10
ingan2.gamma.bmax = 20.0
ingan2.gamma.bounded = True
ingan3 = hs.model.components1D.Lorentzian(centre=2.78,)
ingan3.A.value = 2e7
ingan3.A.bmin = 0.0
ingan3.A.bmax = 1e8
ingan3.A.bounded = True
ingan3.centre.bmin = ingan3.centre.value - 0.05
ingan3.centre.bmax = ingan3.centre.value + 0.05
ingan3.centre.bounded = True
ingan3.gamma.value = 0.1
ingan3.gamma.bmin = -10
ingan3.gamma.bmax = 20.0
ingan3.gamma.bounded = True

In [ ]:
m.extend((bkg, gan1, gan2, ingan1, ingan2, ingan3))

In [ ]:
m.components.Offset.name = 'bkg'
m.components.Lorentzian.name = 'gan1'
m.components.Lorentzian_0.name = 'gan2'
m.components.Lorentzian_1.name = 'ingan1'
m.components.Lorentzian_2.name = 'ingan2'
m.components.Lorentzian_3.name = 'ingan3'

In [ ]:
m.fit(bounded=True)

In [ ]:
m.print_current_values()

In [ ]:
m.plot(plot_components=True)

In [ ]:
m.components.gan1.A.value

In [ ]:
# perform principal component analysis, look at the explained variance
scrop.decomposition(algorithm='svd')
scrop.plot_explained_variance_ratio()

In [ ]:
# take a look at orthogonalized 
scrop.blind_source_separation(number_of_components=8,on_loadings=True,max_iter=1000)
fncn.plot_hs_results(scrop.get_bss_loadings(),scrop.get_bss_factors(),fncn.get_hs_axes(scrop),title='BSS',cmap='viridis')

In [ ]:
np.shape(fncn.get_hs_axes(s)[2])

In [ ]:
np.shape(fncn.get_hs_axes(scrop)[1])

In [ ]:
dh.max()

In [ ]:
span

In [ ]:
span[0]*1e-9

In [ ]:
from spec_im import CLSpectralImage

test_si = CLSpectralImage(fname=fname)

In [ ]:
test_si

In [ ]:
test_si.plot()

In [ ]:
plt.figure()
test_si[355:384].plot()

In [ ]:
plt.figure()
test_si.to_energy()[2.5:2.6].plot()

In [ ]:
test_si.spec_x_array

In [ ]:
dir(si)

In [ ]:
list = ['1','2','3']

In [ ]:
kwarg_print(**{i: ord(i) for i in list})

In [ ]:
def kwarg_print(a='asdf',**kwargs):
    print(a)
    for kw in kwargs.keys():
        print(kw, kwargs[kw])

In [ ]:
esi.dat['se'].shape

In [ ]:
esi.dat['cntr'].shape

In [ ]:
si.dat

In [ ]:
si.plot_ctr('ctr1')

In [ ]:
plt.semilogy(s.get_explained_variance_ratio().data)

In [ ]:
np.searchsorted(np.flip(s.get_explained_variance_ratio()).data,1e-4)

In [ ]:
s.get_explained_variance_ratio().data.shape

In [ ]:
si.dat

In [ ]:
from h5py import File
fh = File(fname)

In [ ]:
tree = h5_to_dictionary(fh, debug=True)

In [ ]:
fh.close()

In [ ]:
tree['measurement']['andor_asi_hyperspec_scan']['spec_map'].shape

In [ ]:
list(fh['app'].items())

In [ ]:
for kk in list(fh['app']['settings'].items()): print(kk[0], kk[1])

In [ ]:
for kk in list(fh['measurement']['andor_asi_hyperspec_scan'].items()): 
    print(kk[0],kk[1])
    if 'HDF5 dataset' in str(kk[1]): 
        print('that is a dataset')

In [ ]:
'a' in 'abc'

In [ ]:
tree.keys()